In [1]:
# module import
import pandas as pd
import os
import matplotlib.pyplot as plt

# path
root_path = '/content/drive/My Drive/DACON/DACON-JEJU'
data_path = os.path.join(root_path, 'data')

In [2]:
# load data
data = pd.read_csv(os.path.join(data_path, '201901-202003.csv'))

# _1_. 기본 사항

* 데이터 스키마
    - REG_YYMM: 년월
    - CARD_SIDO_NM: 카드이용지역-시도(가맹점 주소 기준)
    - CARD_CCG_NM: 카드이용지역-시군구(가맹점 주소 기준)
    - STD_CLSS_NM: 업종명
    - HOM_SIDO_NM: 거주지역-시도(고객 집주소 기준)
    - HOM_CCG_NM: 거주지역-시군구
    - AGE: 연령대
    - SEX_CTGO_CD: 성별
    - FLC: 가구생애주기 (1: 1인가구, 2: 영유아자녀가구, 3: 중고생자녀가구, 4: 성인자녀가구, 5: 노년가구) 
    - CSTMR_CNT: 이용고객수 (명)
    - AMT: 이용금액 (원)
    - CNT: 이용건수 (건)

* 결측치


In [3]:
data.head(10)

,REG_YYMM,CARD_SIDO_NM,CARD_CCG_NM,STD_CLSS_NM,HOM_SIDO_NM,HOM_CCG_NM,AGE,SEX_CTGO_CD,FLC,CSTMR_CNT,AMT,CNT
0,201901,강원,강릉시,건강보조식품 소매업,강원,강릉시,20s,1,1,4,311200,4
1,201901,강원,강릉시,건강보조식품 소매업,강원,강릉시,30s,1,2,7,1374500,8
2,201901,강원,강릉시,건강보조식품 소매업,강원,강릉시,30s,2,2,6,818700,6
3,201901,강원,강릉시,건강보조식품 소매업,강원,강릉시,40s,1,3,4,1717000,5
4,201901,강원,강릉시,건강보조식품 소매업,강원,강릉시,40s,1,4,3,1047300,3
5,201901,강원,강릉시,건강보조식품 소매업,강원,강릉시,40s,2,3,7,627500,7
6,201901,강원,강릉시,건강보조식품 소매업,강원,강릉시,50s,1,4,21,3690830,21
7,201901,강원,강릉시,건강보조식품 소매업,강원,강릉시,50s,2,4,21,3573300,20
8,201901,강원,강릉시,건강보조식품 소매업,강원,강릉시,60s,1,5,19,1433500,20
9,201901,강원,강릉시,건강보조식품 소매업,강원,강릉시,60s,2,5,33,7194600,40


In [4]:
data.tail(10)

,REG_YYMM,CARD_SIDO_NM,CARD_CCG_NM,STD_CLSS_NM,HOM_SIDO_NM,HOM_CCG_NM,AGE,SEX_CTGO_CD,FLC,CSTMR_CNT,AMT,CNT
24697782,202003,충북,충주시,화장품 및 방향제 소매업,충북,충주시,60s,1,5,45,5078720,52
24697783,202003,충북,충주시,화장품 및 방향제 소매업,충북,충주시,60s,2,5,102,9341125,106
24697784,202003,충북,충주시,화장품 및 방향제 소매업,충북,충주시,70s,1,5,8,322630,8
24697785,202003,충북,충주시,화장품 및 방향제 소매업,충북,충주시,70s,2,5,40,3172500,40
24697786,202003,충북,충주시,휴양콘도 운영업,충북,충주시,20s,1,1,3,56000,3
24697787,202003,충북,충주시,휴양콘도 운영업,충북,충주시,30s,1,2,3,43300,4
24697788,202003,충북,충주시,휴양콘도 운영업,충북,충주시,40s,1,3,3,35000,3
24697789,202003,충북,충주시,휴양콘도 운영업,충북,충주시,50s,1,4,4,188000,6
24697790,202003,충북,충주시,휴양콘도 운영업,충북,충주시,50s,2,4,4,99000,6
24697791,202003,충북,충주시,휴양콘도 운영업,충북,충주시,60s,1,5,3,194000,3


In [5]:
# 데이터 구조 확인
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24697792 entries, 0 to 24697791
Data columns (total 12 columns):
 #   Column        Dtype 
---  ------        ----- 
 0   REG_YYMM      int64 
 1   CARD_SIDO_NM  object
 2   CARD_CCG_NM   object
 3   STD_CLSS_NM   object
 4   HOM_SIDO_NM   object
 5   HOM_CCG_NM    object
 6   AGE           object
 7   SEX_CTGO_CD   int64 
 8   FLC           int64 
 9   CSTMR_CNT     int64 
 10  AMT           int64 
 11  CNT           int64 
dtypes: int64(6), object(6)
memory usage: 2.2+ GB


In [15]:
# 컬럼 타입 변경
data['REG_YYMM_NEW'] = data['REG_YYMM'].astype('str')
# data['AGE_NEW'] = data['AGE'].str.replace('s', '').astype('int') # 필요 없을 듯?

In [16]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24697792 entries, 0 to 24697791
Data columns (total 14 columns):
 #   Column        Dtype 
---  ------        ----- 
 0   REG_YYMM      int64 
 1   CARD_SIDO_NM  object
 2   CARD_CCG_NM   object
 3   STD_CLSS_NM   object
 4   HOM_SIDO_NM   object
 5   HOM_CCG_NM    object
 6   AGE           object
 7   SEX_CTGO_CD   int64 
 8   FLC           int64 
 9   CSTMR_CNT     int64 
 10  AMT           int64 
 11  CNT           int64 
 12  REG_YYMM_NEW  object
 13  AGE_NEW       int64 
dtypes: int64(7), object(7)
memory usage: 2.6+ GB


In [17]:
# 결측치
data.isnull().sum()

REG_YYMM             0
CARD_SIDO_NM         0
CARD_CCG_NM      87213
STD_CLSS_NM          0
HOM_SIDO_NM          0
HOM_CCG_NM      147787
AGE                  0
SEX_CTGO_CD          0
FLC                  0
CSTMR_CNT            0
AMT                  0
CNT                  0
REG_YYMM_NEW         0
AGE_NEW              0
dtype: int64

 고객 측에서 결측치는 의미 없을 것 같고, 어느 시도, 어느 업종에서 결측치가 있는지 봐야 할 것 같다.

In [7]:
def display_null_df(df, null_col, check_col, *group_standards):
    null_df  = df[df[null_col].isnull()].groupby(list(group_standards), as_index=False).count()
    for col in null_df[check_col].unique():
        null_df[null_df[check_col] == col]
        display(null_df[null_df[check_col] == col])
        print('')
    return f"그룹핑 컬럼: {group_standards}, 확인 컬럼: {check_col}"

In [8]:
# 카드 시군구 명 결측치 확인
display_null_df(data, 'CARD_CCG_NM', 'REG_YYMM', 'REG_YYMM', 'CARD_SIDO_NM', 'STD_CLSS_NM')

,REG_YYMM,CARD_SIDO_NM,STD_CLSS_NM,CARD_CCG_NM,HOM_SIDO_NM,HOM_CCG_NM,AGE,SEX_CTGO_CD,FLC,CSTMR_CNT,AMT,CNT
0,201901,세종,건강보조식품 소매업,0,10,1,10,10,10,10,10,10
1,201901,세종,골프장 운영업,0,44,38,44,44,44,44,44,44
2,201901,세종,과실 및 채소 소매업,0,127,107,127,127,127,127,127,127
3,201901,세종,관광 민예품 및 선물용품 소매업,0,1,0,1,1,1,1,1,1
4,201901,세종,그외 기타 종합 소매업,0,37,26,37,37,37,37,37,37
5,201901,세종,기타 대형 종합 소매업,0,415,391,415,415,415,415,415,415
6,201901,세종,기타 수상오락 서비스업,0,1,0,1,1,1,1,1,1
7,201901,세종,기타 외국식 음식점업,0,81,57,81,81,81,81,81,81
8,201901,세종,기타 주점업,0,5,0,5,5,5,5,5,5
9,201901,세종,기타음식료품위주종합소매업,0,137,113,137,137,137,137,137,137


,REG_YYMM,CARD_SIDO_NM,STD_CLSS_NM,CARD_CCG_NM,HOM_SIDO_NM,HOM_CCG_NM,AGE,SEX_CTGO_CD,FLC,CSTMR_CNT,AMT,CNT
30,201902,세종,건강보조식품 소매업,0,9,0,9,9,9,9,9,9
31,201902,세종,골프장 운영업,0,52,45,52,52,52,52,52,52
32,201902,세종,과실 및 채소 소매업,0,120,100,120,120,120,120,120,120
33,201902,세종,관광 민예품 및 선물용품 소매업,0,1,0,1,1,1,1,1,1
34,201902,세종,그외 기타 종합 소매업,0,50,39,50,50,50,50,50,50
35,201902,세종,기타 대형 종합 소매업,0,399,375,399,399,399,399,399,399
36,201902,세종,기타 외국식 음식점업,0,70,48,70,70,70,70,70,70
37,201902,세종,기타 주점업,0,3,0,3,3,3,3,3,3
38,201902,세종,기타음식료품위주종합소매업,0,146,123,146,146,146,146,146,146
39,201902,세종,마사지업,0,3,0,3,3,3,3,3,3


,REG_YYMM,CARD_SIDO_NM,STD_CLSS_NM,CARD_CCG_NM,HOM_SIDO_NM,HOM_CCG_NM,AGE,SEX_CTGO_CD,FLC,CSTMR_CNT,AMT,CNT
59,201903,세종,건강보조식품 소매업,0,9,0,9,9,9,9,9,9
60,201903,세종,골프장 운영업,0,105,97,105,105,105,105,105,105
61,201903,세종,과실 및 채소 소매업,0,134,113,134,134,134,134,134,134
62,201903,세종,관광 민예품 및 선물용품 소매업,0,1,0,1,1,1,1,1,1
63,201903,세종,그외 기타 종합 소매업,0,50,38,50,50,50,50,50,50
64,201903,세종,기타 대형 종합 소매업,0,447,423,447,447,447,447,447,447
65,201903,세종,기타 외국식 음식점업,0,98,76,98,98,98,98,98,98
66,201903,세종,기타 주점업,0,2,0,2,2,2,2,2,2
67,201903,세종,기타음식료품위주종합소매업,0,142,118,142,142,142,142,142,142
68,201903,세종,마사지업,0,4,0,4,4,4,4,4,4


,REG_YYMM,CARD_SIDO_NM,STD_CLSS_NM,CARD_CCG_NM,HOM_SIDO_NM,HOM_CCG_NM,AGE,SEX_CTGO_CD,FLC,CSTMR_CNT,AMT,CNT
88,201904,세종,건강보조식품 소매업,0,11,0,11,11,11,11,11,11
89,201904,세종,골프장 운영업,0,106,97,106,106,106,106,106,106
90,201904,세종,과실 및 채소 소매업,0,141,121,141,141,141,141,141,141
91,201904,세종,관광 민예품 및 선물용품 소매업,0,4,0,4,4,4,4,4,4
92,201904,세종,그외 기타 종합 소매업,0,54,43,54,54,54,54,54,54
93,201904,세종,기타 대형 종합 소매업,0,395,371,395,395,395,395,395,395
94,201904,세종,기타 외국식 음식점업,0,95,73,95,95,95,95,95,95
95,201904,세종,기타 주점업,0,3,0,3,3,3,3,3,3
96,201904,세종,기타음식료품위주종합소매업,0,144,121,144,144,144,144,144,144
97,201904,세종,마사지업,0,3,0,3,3,3,3,3,3


,REG_YYMM,CARD_SIDO_NM,STD_CLSS_NM,CARD_CCG_NM,HOM_SIDO_NM,HOM_CCG_NM,AGE,SEX_CTGO_CD,FLC,CSTMR_CNT,AMT,CNT
117,201905,세종,건강보조식품 소매업,0,10,0,10,10,10,10,10,10
118,201905,세종,골프장 운영업,0,110,102,110,110,110,110,110,110
119,201905,세종,과실 및 채소 소매업,0,151,130,151,151,151,151,151,151
120,201905,세종,관광 민예품 및 선물용품 소매업,0,11,4,11,11,11,11,11,11
121,201905,세종,그외 기타 분류안된 오락관련 서비스업,0,2,1,2,2,2,2,2,2
122,201905,세종,그외 기타 종합 소매업,0,52,38,52,52,52,52,52,52
123,201905,세종,기타 대형 종합 소매업,0,419,395,419,419,419,419,419,419
124,201905,세종,기타 외국식 음식점업,0,96,73,96,96,96,96,96,96
125,201905,세종,기타 주점업,0,5,0,5,5,5,5,5,5
126,201905,세종,기타음식료품위주종합소매업,0,180,156,180,180,180,180,180,180


...

,REG_YYMM,CARD_SIDO_NM,STD_CLSS_NM,CARD_CCG_NM,HOM_SIDO_NM,HOM_CCG_NM,AGE,SEX_CTGO_CD,FLC,CSTMR_CNT,AMT,CNT
321,201912,세종,건강보조식품 소매업,0,7,0,7,7,7,7,7,7
322,201912,세종,골프장 운영업,0,75,67,75,75,75,75,75,75
323,201912,세종,과실 및 채소 소매업,0,119,97,119,119,119,119,119,119
324,201912,세종,관광 민예품 및 선물용품 소매업,0,9,2,9,9,9,9,9,9
325,201912,세종,그외 기타 종합 소매업,0,26,17,26,26,26,26,26,26
326,201912,세종,기타 대형 종합 소매업,0,393,369,393,393,393,393,393,393
327,201912,세종,기타 외국식 음식점업,0,105,81,105,105,105,105,105,105
328,201912,세종,기타 주점업,0,4,0,4,4,4,4,4,4
329,201912,세종,기타음식료품위주종합소매업,0,137,113,137,137,137,137,137,137
330,201912,세종,마사지업,0,4,0,4,4,4,4,4,4


,REG_YYMM,CARD_SIDO_NM,STD_CLSS_NM,CARD_CCG_NM,HOM_SIDO_NM,HOM_CCG_NM,AGE,SEX_CTGO_CD,FLC,CSTMR_CNT,AMT,CNT
350,202001,세종,건강보조식품 소매업,0,10,0,10,10,10,10,10,10
351,202001,세종,골프장 운영업,0,49,45,49,49,49,49,49,49
352,202001,세종,과실 및 채소 소매업,0,142,121,142,142,142,142,142,142
353,202001,세종,관광 민예품 및 선물용품 소매업,0,11,5,11,11,11,11,11,11
354,202001,세종,그외 기타 스포츠시설 운영업,0,1,0,1,1,1,1,1,1
355,202001,세종,그외 기타 종합 소매업,0,44,35,44,44,44,44,44,44
356,202001,세종,기타 대형 종합 소매업,0,414,390,414,414,414,414,414,414
357,202001,세종,기타 외국식 음식점업,0,88,65,88,88,88,88,88,88
358,202001,세종,기타 주점업,0,5,1,5,5,5,5,5,5
359,202001,세종,기타음식료품위주종합소매업,0,156,132,156,156,156,156,156,156


,REG_YYMM,CARD_SIDO_NM,STD_CLSS_NM,CARD_CCG_NM,HOM_SIDO_NM,HOM_CCG_NM,AGE,SEX_CTGO_CD,FLC,CSTMR_CNT,AMT,CNT
380,202002,세종,건강보조식품 소매업,0,10,0,10,10,10,10,10,10
381,202002,세종,골프장 운영업,0,64,57,64,64,64,64,64,64
382,202002,세종,과실 및 채소 소매업,0,132,110,132,132,132,132,132,132
383,202002,세종,관광 민예품 및 선물용품 소매업,0,1,0,1,1,1,1,1,1
384,202002,세종,그외 기타 종합 소매업,0,22,14,22,22,22,22,22,22
385,202002,세종,기타 대형 종합 소매업,0,316,292,316,316,316,316,316,316
386,202002,세종,기타 외국식 음식점업,0,80,56,80,80,80,80,80,80
387,202002,세종,기타 주점업,0,5,1,5,5,5,5,5,5
388,202002,세종,기타음식료품위주종합소매업,0,142,118,142,142,142,142,142,142
389,202002,세종,마사지업,0,1,0,1,1,1,1,1,1


,REG_YYMM,CARD_SIDO_NM,STD_CLSS_NM,CARD_CCG_NM,HOM_SIDO_NM,HOM_CCG_NM,AGE,SEX_CTGO_CD,FLC,CSTMR_CNT,AMT,CNT
409,202003,세종,건강보조식품 소매업,0,9,0,9,9,9,9,9,9
410,202003,세종,골프장 운영업,0,88,80,88,88,88,88,88,88
411,202003,세종,과실 및 채소 소매업,0,126,105,126,126,126,126,126,126
412,202003,세종,그외 기타 종합 소매업,0,25,13,25,25,25,25,25,25
413,202003,세종,기타 대형 종합 소매업,0,349,325,349,349,349,349,349,349
414,202003,세종,기타 외국식 음식점업,0,72,51,72,72,72,72,72,72
415,202003,세종,기타 주점업,0,2,0,2,2,2,2,2,2
416,202003,세종,기타음식료품위주종합소매업,0,159,135,159,159,159,159,159,159
417,202003,세종,마사지업,0,3,0,3,3,3,3,3,3
418,202003,세종,비알콜 음료점업,0,268,244,268,268,268,268,268,268


"그룹핑 컬럼: ('REG_YYMM', 'CARD_SIDO_NM', 'STD_CLSS_NM'), 확인 컬럼: REG_YYMM"

In [11]:
# 고객 거주지역 별 결측치 확인: 굉장히 다양한데?
display_null_df(data, 'HOM_CCG_NM', 'REG_YYMM', 'REG_YYMM', 'CARD_SIDO_NM')

,REG_YYMM,CARD_SIDO_NM,CARD_CCG_NM,STD_CLSS_NM,HOM_SIDO_NM,HOM_CCG_NM,AGE,SEX_CTGO_CD,FLC,CSTMR_CNT,AMT,CNT
0,201901,강원,352,352,352,0,352,352,352,352,352,352
1,201901,경기,1202,1202,1202,0,1202,1202,1202,1202,1202,1202
2,201901,경남,163,163,163,0,163,163,163,163,163,163
3,201901,경북,304,304,304,0,304,304,304,304,304,304
4,201901,광주,69,69,69,0,69,69,69,69,69,69
5,201901,대구,128,128,128,0,128,128,128,128,128,128
6,201901,대전,1304,1304,1304,0,1304,1304,1304,1304,1304,1304
7,201901,부산,315,315,315,0,315,315,315,315,315,315
8,201901,서울,1928,1928,1928,0,1928,1928,1928,1928,1928,1928
9,201901,세종,0,515,515,0,515,515,515,515,515,515


,REG_YYMM,CARD_SIDO_NM,CARD_CCG_NM,STD_CLSS_NM,HOM_SIDO_NM,HOM_CCG_NM,AGE,SEX_CTGO_CD,FLC,CSTMR_CNT,AMT,CNT
17,201902,강원,295,295,295,0,295,295,295,295,295,295
18,201902,경기,1211,1211,1211,0,1211,1211,1211,1211,1211,1211
19,201902,경남,195,195,195,0,195,195,195,195,195,195
20,201902,경북,349,349,349,0,349,349,349,349,349,349
21,201902,광주,104,104,104,0,104,104,104,104,104,104
22,201902,대구,156,156,156,0,156,156,156,156,156,156
23,201902,대전,1294,1294,1294,0,1294,1294,1294,1294,1294,1294
24,201902,부산,352,352,352,0,352,352,352,352,352,352
25,201902,서울,1938,1938,1938,0,1938,1938,1938,1938,1938,1938
26,201902,세종,0,505,505,0,505,505,505,505,505,505


,REG_YYMM,CARD_SIDO_NM,CARD_CCG_NM,STD_CLSS_NM,HOM_SIDO_NM,HOM_CCG_NM,AGE,SEX_CTGO_CD,FLC,CSTMR_CNT,AMT,CNT
34,201903,강원,206,206,206,0,206,206,206,206,206,206
35,201903,경기,1261,1261,1261,0,1261,1261,1261,1261,1261,1261
36,201903,경남,180,180,180,0,180,180,180,180,180,180
37,201903,경북,346,346,346,0,346,346,346,346,346,346
38,201903,광주,91,91,91,0,91,91,91,91,91,91
39,201903,대구,120,120,120,0,120,120,120,120,120,120
40,201903,대전,1303,1303,1303,0,1303,1303,1303,1303,1303,1303
41,201903,부산,288,288,288,0,288,288,288,288,288,288
42,201903,서울,1954,1954,1954,0,1954,1954,1954,1954,1954,1954
43,201903,세종,0,515,515,0,515,515,515,515,515,515


,REG_YYMM,CARD_SIDO_NM,CARD_CCG_NM,STD_CLSS_NM,HOM_SIDO_NM,HOM_CCG_NM,AGE,SEX_CTGO_CD,FLC,CSTMR_CNT,AMT,CNT
51,201904,강원,184,184,184,0,184,184,184,184,184,184
52,201904,경기,1199,1199,1199,0,1199,1199,1199,1199,1199,1199
53,201904,경남,194,194,194,0,194,194,194,194,194,194
54,201904,경북,321,321,321,0,321,321,321,321,321,321
55,201904,광주,92,92,92,0,92,92,92,92,92,92
56,201904,대구,113,113,113,0,113,113,113,113,113,113
57,201904,대전,1311,1311,1311,0,1311,1311,1311,1311,1311,1311
58,201904,부산,253,253,253,0,253,253,253,253,253,253
59,201904,서울,1905,1905,1905,0,1905,1905,1905,1905,1905,1905
60,201904,세종,0,509,509,0,509,509,509,509,509,509


,REG_YYMM,CARD_SIDO_NM,CARD_CCG_NM,STD_CLSS_NM,HOM_SIDO_NM,HOM_CCG_NM,AGE,SEX_CTGO_CD,FLC,CSTMR_CNT,AMT,CNT
68,201905,강원,248,248,248,0,248,248,248,248,248,248
69,201905,경기,1372,1372,1372,0,1372,1372,1372,1372,1372,1372
70,201905,경남,188,188,188,0,188,188,188,188,188,188
71,201905,경북,347,347,347,0,347,347,347,347,347,347
72,201905,광주,112,112,112,0,112,112,112,112,112,112
73,201905,대구,126,126,126,0,126,126,126,126,126,126
74,201905,대전,1328,1328,1328,0,1328,1328,1328,1328,1328,1328
75,201905,부산,328,328,328,0,328,328,328,328,328,328
76,201905,서울,2046,2046,2046,0,2046,2046,2046,2046,2046,2046
77,201905,세종,0,529,529,0,529,529,529,529,529,529


...

,REG_YYMM,CARD_SIDO_NM,CARD_CCG_NM,STD_CLSS_NM,HOM_SIDO_NM,HOM_CCG_NM,AGE,SEX_CTGO_CD,FLC,CSTMR_CNT,AMT,CNT
187,201912,강원,257,257,257,0,257,257,257,257,257,257
188,201912,경기,1241,1241,1241,0,1241,1241,1241,1241,1241,1241
189,201912,경남,164,164,164,0,164,164,164,164,164,164
190,201912,경북,346,346,346,0,346,346,346,346,346,346
191,201912,광주,97,97,97,0,97,97,97,97,97,97
192,201912,대구,115,115,115,0,115,115,115,115,115,115
193,201912,대전,1297,1297,1297,0,1297,1297,1297,1297,1297,1297
194,201912,부산,320,320,320,0,320,320,320,320,320,320
195,201912,서울,2036,2036,2036,0,2036,2036,2036,2036,2036,2036
196,201912,세종,0,523,523,0,523,523,523,523,523,523


,REG_YYMM,CARD_SIDO_NM,CARD_CCG_NM,STD_CLSS_NM,HOM_SIDO_NM,HOM_CCG_NM,AGE,SEX_CTGO_CD,FLC,CSTMR_CNT,AMT,CNT
204,202001,강원,371,371,371,0,371,371,371,371,371,371
205,202001,경기,1199,1199,1199,0,1199,1199,1199,1199,1199,1199
206,202001,경남,194,194,194,0,194,194,194,194,194,194
207,202001,경북,380,380,380,0,380,380,380,380,380,380
208,202001,광주,104,104,104,0,104,104,104,104,104,104
209,202001,대구,143,143,143,0,143,143,143,143,143,143
210,202001,대전,1310,1310,1310,0,1310,1310,1310,1310,1310,1310
211,202001,부산,318,318,318,0,318,318,318,318,318,318
212,202001,서울,1924,1924,1924,0,1924,1924,1924,1924,1924,1924
213,202001,세종,0,523,523,0,523,523,523,523,523,523


,REG_YYMM,CARD_SIDO_NM,CARD_CCG_NM,STD_CLSS_NM,HOM_SIDO_NM,HOM_CCG_NM,AGE,SEX_CTGO_CD,FLC,CSTMR_CNT,AMT,CNT
221,202002,강원,212,212,212,0,212,212,212,212,212,212
222,202002,경기,838,838,838,0,838,838,838,838,838,838
223,202002,경남,104,104,104,0,104,104,104,104,104,104
224,202002,경북,184,184,184,0,184,184,184,184,184,184
225,202002,광주,50,50,50,0,50,50,50,50,50,50
226,202002,대구,60,60,60,0,60,60,60,60,60,60
227,202002,대전,1133,1133,1133,0,1133,1133,1133,1133,1133,1133
228,202002,부산,161,161,161,0,161,161,161,161,161,161
229,202002,서울,1536,1536,1536,0,1536,1536,1536,1536,1536,1536
230,202002,세종,0,505,505,0,505,505,505,505,505,505


,REG_YYMM,CARD_SIDO_NM,CARD_CCG_NM,STD_CLSS_NM,HOM_SIDO_NM,HOM_CCG_NM,AGE,SEX_CTGO_CD,FLC,CSTMR_CNT,AMT,CNT
238,202003,강원,82,82,82,0,82,82,82,82,82,82
239,202003,경기,772,772,772,0,772,772,772,772,772,772
240,202003,경남,74,74,74,0,74,74,74,74,74,74
241,202003,경북,103,103,103,0,103,103,103,103,103,103
242,202003,광주,47,47,47,0,47,47,47,47,47,47
243,202003,대구,22,22,22,0,22,22,22,22,22,22
244,202003,대전,1069,1069,1069,0,1069,1069,1069,1069,1069,1069
245,202003,부산,79,79,79,0,79,79,79,79,79,79
246,202003,서울,1337,1337,1337,0,1337,1337,1337,1337,1337,1337
247,202003,세종,0,489,489,0,489,489,489,489,489,489


"그룹핑 컬럼: ('REG_YYMM', 'CARD_SIDO_NM'), 확인 컬럼: REG_YYMM"

# _2_. 데이터 분포 확인
* 요약 통계량
* 이상치
* 히스토그램
* 산점도


In [40]:
# 변수별 분류
cat_features = ['REG_YYMM_NEW', 'CARD_SIDO_NM', 'CARD_CCG_NM', 'STD_CLSS_NM', 'HOM_SIDO_NM', 'HOM_CCG_NM', 'AGE', 'SEX_CTGO_CD', 'FLC']
num_features = ['CSTMR_CNT', 'AMT', 'CNT']

In [33]:
# 통계량 확인
for feature in cat_features:
    print(feature)
    display(pd.DataFrame([data[feature].value_counts(), (data[feature].value_counts() / len(data[feature]))*100]).T)
    print('')

REG_YYMM_NEW


,REG_YYMM_NEW,REG_YYMM_NEW
201908,1880296.0,7.613215
201905,1772237.0,7.175690
201910,1751005.0,7.089723
201909,1734914.0,7.024571
201907,1721353.0,6.969664
201906,1712696.0,6.934612
201912,1674381.0,6.779476
202001,1672889.0,6.773435
201903,1668675.0,6.756373
201904,1655655.0,6.703656



CARD_SIDO_NM


,CARD_SIDO_NM,CARD_SIDO_NM
서울,6259502.0,25.344379
경기,5188230.0,21.006858
부산,2030810.0,8.222638
인천,1278410.0,5.176212
강원,1230355.0,4.981640
경남,1222818.0,4.951123
경북,1204831.0,4.878294
충남,1178159.0,4.770301
충북,834707.0,3.379683
전북,829466.0,3.358462



CARD_CCG_NM


...

,AGE,AGE
40s,5802447.0,23.493788
30s,5550519.0,22.473746
20s,5091675.0,20.615912
50s,4455687.0,18.040831
60s,2606168.0,10.552231
70s,876622.0,3.549394
10s,314674.0,1.274098



SEX_CTGO_CD


,SEX_CTGO_CD,SEX_CTGO_CD
1,14506378.0,58.735526
2,10191414.0,41.264474



FLC


,FLC,FLC
2,6188801.0,25.058115
1,5526140.0,22.375037
4,5445320.0,22.047801
3,4054741.0,16.417423
5,3482790.0,14.101625


In [31]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24697792 entries, 0 to 24697791
Data columns (total 14 columns):
 #   Column        Dtype 
---  ------        ----- 
 0   REG_YYMM      int64 
 1   CARD_SIDO_NM  object
 2   CARD_CCG_NM   object
 3   STD_CLSS_NM   object
 4   HOM_SIDO_NM   object
 5   HOM_CCG_NM    object
 6   AGE           object
 7   SEX_CTGO_CD   int64 
 8   FLC           int64 
 9   CSTMR_CNT     int64 
 10  AMT           int64 
 11  CNT           int64 
 12  REG_YYMM_NEW  object
 13  AGE_NEW       int64 
dtypes: int64(7), object(7)
memory usage: 2.6+ GB


In [42]:
# 숫자형 데이터 요약 통계량 확인
data[num_features + ['AGE_NEW', 'FLC']].describe().astype('int64')

,CSTMR_CNT,AMT,CNT,AGE_NEW,FLC
count,24697792,24697792,24697792,24697792,24697792
mean,61,2441352,126,38,2
std,355,17260959,966,14,1
min,3,20,1,10,1
25%,4,90200,5,30,2
50%,8,252680,12,40,3
75%,24,843337,40,50,4
max,32813,2349523701,120357,70,5


# _3_.  고객 구매금액 별로?
